In [1]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact

proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"

raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)

project root: /Users/joachimpfefferkorn/repos/neurovolume


GPT:

In [2]:
bold_image = ants.image_read(bold_stim_filepath)  # Replace with your BOLD image path
t1_image = ants.image_read(t1_input_filepath)     # Replace with your T1 image path


# If the BOLD image is 4D, extract the mean image for registration
if bold_image.dimension == 4:
    bold_mean_np = bold_image.numpy().mean(axis=-1)  # Compute the mean along the time axis
    bold_mean = ants.from_numpy(bold_mean_np, spacing=bold_image.spacing[:3])
else:
    bold_mean = bold_image

# Perform registration
registration = ants.registration(
    fixed=t1_image,
    moving=bold_mean,
    type_of_transform='Rigid'  # You can also use 'Affine' or 'SyN' for deformable registration
)

In [3]:
print(f"BOLD image dimensions: {bold_image.dimension}, shape: {bold_image.shape}")
print(f"T1 image dimensions: {t1_image.dimension}, shape: {t1_image.shape}")
print(f"Forward Transforms: {registration['fwdtransforms']}")

BOLD image dimensions: 4, shape: (64, 64, 35, 185)
T1 image dimensions: 3, shape: (512, 512, 296)
Forward Transforms: ['/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpq30xmcav0GenericAffine.mat']


In [4]:
bold_mean_registered = ants.apply_transforms(
    fixed=t1_image,
    moving=bold_mean,
    transformlist=registration['fwdtransforms'],
    interpolator='linear'
)
ants.image_write(bold_mean_registered, 'bold_mean_registered.nii.gz')


In [5]:
print(bold_mean_registered)
explore_3D_vol(bold_mean_registered.numpy())

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 296)
	 Spacing    : (0.4785, 0.4785, 0.5)
	 Origin     : (119.989, 104.52, -84.2457)
	 Direction  : [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]



interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

THIS WORKS!

In [ ]:
# Apply the transformation to the full BOLD sequence
bold_registered = ants.apply_transforms(
    fixed=t1_image,
    moving=bold_image,
    transformlist=registration['fwdtransforms'],
    interpolator='linear',
    imagetype=3  # Specify imagetype=3 for 4D images
)

In [ ]:
# Save the registered BOLD image
ants.image_write(bold_registered, 'bold_registered_to_t1.nii.gz')

In [ ]:
#I have a hunch this is correct but let's see:
spacing = (1/raw_t1_img.spacing[0], 1/raw_t1_img.spacing[1], 1/raw_t1_img.spacing[2])
print(spacing)

In [ ]:
bold_frame_img = ants.from_numpy(raw_stim_bold.numpy()[:,:,:,1])
#bold_resampled = ants.resample_image(bold_frame_img, spacing, use_voxels=False)
bold_resampled = ants.resample_image(bold_frame_img, spacing, use_voxels=False)

registered_frame = ants.registration(
    fixed=mni_img,
    moving=bold_resampled,
    type_of_transform='SyNBold',
    verbose=False
)

In [ ]:
print(registered_frame['fwdtransforms'])

In [ ]:
attempted_bold = ants.apply_transforms(mni_img, bold_resampled, registered_frame['fwdtransforms'], imagetype=1)


In [ ]:
explore_3D_vol(attempted_bold.numpy())